# Zapytania złożone

Przed rozpoczęciem pracy konieczne jest skonfigurowanie bazy oraz załadowanie danych:
```ruby
$:.unshift "."
require 'db_setup'
require 'db_load'
```

In [1]:
$:.unshift "."
require 'db_setup'
require 'db_load'

==  AddIndicesMigration: migrating ============================================
==  AddIndicesMigration: migrated (0.0000s) ===================================



true

W dalszych zadaniach będzie wykonywać polecenia korzystając z następujących klas, zmapowanych na odpowiadające im 
table w bazie danych:
```ruby
class Author < ActiveRecord::Base
  # name      (string)
  # surname   (string)
  # born      (datetime) 
  # died      (datetime)
  # image_url (string)
  
  has_many :books
end

class Book < ActiveRecord::Base
  # title     (string)
  # author    (Author)
  # published (integer)

  belongs_to :author
end
```

ActiveRecord udostępnia interfejs pozwalający wyszukiwać dane w bazie również z wykorzystaniem związków zdefiniowanych pomiędzy
klasami. Do definiowania tych związków służą polecenia `has_many`, `belongs_to`, `has_one` oraz `has_and_belongs_to_many`.
Przykładowo jeśli chcemy dowiedzieć się kto był autorem pierwszej książki wykonujemy zapytanie:
```ruby
book = Book.first
puts book.title
puts book.author
```

In [2]:
book = Book.first
puts book.title
puts book.author

Végjáték (regény)
#<Author:0x000000020ed740>


Niestety prezentowany wynik nie jest zbyt użyteczny. Ponieważ jednak metoda `author` zwraca obiekt, możemy na nim wywoływać 
metody - tak samo jak w przypadku autora załadowanego bezpośrednio z wykorzystaniem metod zdefiniowanych dla klasy `Author`
```ruby
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"
```

In [61]:
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"

Végjáték (regény)
Orson Scott Card


Co więcej, jeśli mamy do czynienia ze związkiem jeden-do-wiele bądź wiele-do-wiele, możemy wywyływać wszystkie dotychczas poznane 
metody na kolekcji powiązanych obiektów. Istotnie ułatwia to wyszukiwanie potrzebnych danych:
```ruby
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end
```

In [4]:
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end

Anna Karenina
Azazel (powieść)
Anna Karenina


[#<Book id: 33, title: "Anna Karenina", language: nil, author_id: 35, published: 1877, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

Różnica pomiędzy tymi zapytaniami polega na tym, że w pierwszym przypadku szukamy książki, których tytuł zaczyna się na "A" 
w całym zbiorze książek, a w drugi przypadku wśród książek napisanych przez Tołstoja.

Aby umożliwić formatowanie wyników zapytania, IRuby udostępnia metodę `IRuby.html`, która pozwala na wyświetlanie kodu
HTML. Przykładowo
```ruby
IRuby.html "<h1>Test HTML-a</h1>"
```

In [2]:
IRuby.html "<h1>Test HTML-a</h1>"

"<h1>Test HTML-a</h1>"

Dzięki temu będziemy mogli formatować wyniki zapytań, a wszczególności wyświetlać obrazy przedstwiające autorów oraz książki. 
Należy jednak wziąć pod uwagę, że nie wszystkie książki oraz nie wszyscy autorzy posiadają odpowiadające im obrazki.
Aby wyświetlić imię, nazwisko oraz obraz pierwszego autora piszemy:
```ruby
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
```

In [6]:
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"

"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

Możemy również zdefniować funkcję, np. `display_author`, której zadaniem będzie wyświetlenie autora w tej postaci. 
Jedyny problem polega na tym, że funkcja `IRuby.html` może być wywoływana tylko raz. Dlatego jeśli chcemy wyświetlić kilku autorów,
musimy połączyć odpowiadające im kody HTML i wynik przekazać do funkcji `html`
```ruby
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document
```

In [31]:
def display_author(author)
  "<h1>#{author.name} #{author.surname} #{author.born} - #{author.died} </h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document

"<h1>Orson Scott Card 1951-08-24 -  </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><h1>Andrzej Sapkowski 1948-06-21 -  </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/>"

## Zadania

### Zadanie 1

Wyświetl wszystkich autorów, którzy urodzili się przed rokiem 1900.

In [3]:
author = Author.where(born: ("0".."1899-12-31")) 
author.each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end

Juliusz Słowacki 1809-09-04
Henryk Sienkiewicz 1846-05-05
Eliza Orzeszkowa 1841-06-06
Adam Mickiewicz 1798-12-24
Jack London 1876-01-12
Mikołaj Kopernik 1473-02-19
Maria Franciszka Kozłowska 1862-05-27
William Szekspir 1564-04-23
John Ronald Reuel Tolkien 1892-01-03
Aleksander Dumas (ojciec) 1802-07-24
Tadeusz Dołęga-Mostowicz 1898-08-10
Vladimir Nabokov 1899-04-22
Adolf Hitler 1889-04-20
Benedykt Chmielowski 1700-03-20
Aldous Huxley 1894-07-26
Honoré de Balzac 1799-05-20
Franz Kafka 1883-07-03
Howard Phillips Lovecraft 1890-08-20
Lew Tołstoj 1828-09-09
Ilja Ilf 1897-10-15
Gabriela Zapolska 1857-03-30
Clive Staples Lewis 1898-11-29
Iwan Turgieniew 1818-11-09
Lucy Maud Montgomery 1874-11-30
Aleksander Majkowski 1876-07-17


[#<Author id: 3, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 4, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-05", image_url: nil>, #<Author id: 5, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-18", image_url: nil>, #<Author id: 10, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 13, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 17, name: "Mikołaj", surname: "Kopernik", born: "1473-02-19", died: "1543-05-24", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 18, name: "Maria Franciszka", surname: "Kozłowska", born: "1862-05-27", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 20, name: "William", surname: "Szekspir", born: "156

### Zadanie 2

Zmodyfikuj funkcję wyświetlającą autorów tak aby obejmowała ich datę urodzin i śmierci w formacie `dzień.miesiąc.rok`. Następnie wyświetl wszystkich autorów,
którzy zmarli w XX wieku.

In [29]:
author = Author.where(died: ("1900-1-1".."2000-12-31")) 
author.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d.%m.%Y")} - #{author.died.strftime("%d.%m.%Y")}"
end

Henryk Sienkiewicz 05.05.1846 - 05.11.1916
Eliza Orzeszkowa 06.06.1841 - 18.05.1910
Jack London 12.01.1876 - 22.11.1916
Antoni Gołubiew 25.02.1907 - 27.06.1979
George Orwell 25.06.1903 - 21.01.1950
John Ronald Reuel Tolkien 03.01.1892 - 02.09.1973
Tadeusz Dołęga-Mostowicz 10.08.1898 - 20.09.1939
Vladimir Nabokov 22.04.1899 - 02.07.1977
Adolf Hitler 20.04.1889 - 30.04.1945
Aldous Huxley 26.07.1894 - 22.11.1963
Franz Kafka 03.07.1883 - 03.06.1924
Howard Phillips Lovecraft 20.08.1890 - 15.03.1937
Karol Olgierd Borchardt 25.03.1905 - 20.05.1986
Lew Tołstoj 09.09.1828 - 20.11.1910
Janusz Zajdel 15.08.1938 - 19.07.1985
Isaac Asimov 04.10.1919 - 06.04.1992
Ilja Ilf 15.10.1897 - 13.04.1937
Jewgienij Pietrow (pisarz) 13.12.1903 - 02.07.1942
Gabriela Zapolska 30.03.1857 - 17.12.1921
Clive Staples Lewis 29.11.1898 - 22.11.1963
Lucy Maud Montgomery 30.11.1874 - 24.04.1942
Alfred Szklarski 21.01.1912 - 09.04.1992
Aleksander Majkowski 17.07.1876 - 10.02.1938
Ian Fleming 28.05.1908 - 12.08.1964


[#<Author id: 4, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-05", image_url: nil>, #<Author id: 5, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-18", image_url: nil>, #<Author id: 13, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 16, name: "Antoni", surname: "Gołubiew", born: "1907-02-25", died: "1979-06-27", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 19, name: "George", surname: "Orwell", born: "1903-06-25", died: "1950-01-21", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 22, name: "John Ronald Reuel", surname: "Tolkien", born: "1892-01-03", died: "1973-09-02", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 24, name: "Tadeusz", surname: "Dołęga-Mostowicz", born: "1898-08-10", died: "1939-09-20", image_url: "http

### Zadanie 3

Wyświetl 5 najmłodszych autorów, którzy posiadają przypisane zdjęcie.

In [51]:
author = Author.where("image_url is not null").order(:born).last(5) 
author.each do |author|
    puts "#{author.name} #{author.surname} #{author.born.strftime("%d.%m.%Y")}"
end

Orson Scott Card 24.08.1951
Boris Akunin 20.05.1956
Rafał A. Ziemkiewicz 13.09.1964
J. K. Rowling 31.07.1965
Andrzej Pilipiuk 20.03.1974


[#<Author id: 11, name: "Orson Scott", surname: "Card", born: "1951-08-24", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 36, name: "Boris", surname: "Akunin", born: "1956-05-20", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 55, name: "Rafał A.", surname: "Ziemkiewicz", born: "1964-09-13", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 21, name: "J. K.", surname: "Rowling", born: "1965-07-31", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 46, name: "Andrzej", surname: "Pilipiuk", born: "1974-03-20", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

### Zadanie 4

Zdefiniuj funkcję `display_book`, która będzie zwracała fragment kodu HTML obejmujący tytuł książki, rok jej wydania oraz
przypisany obrazek.
Wyświetl najwcześniej wydaną książkę.

In [67]:
def display_book(book)
    "<h1>#{book.title} #{book.published}</h1><img src='#{book.author.image_url}'/>"
end
book = Book.where("published is not null and published != 0").order(:published).first
document = display_book(book)
IRuby.html document

"<h1>De revolutionibus orbium coelestium 1543</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300'/>"

### Zadanie 5

Wyświetl wszystkie książki, które nie mają przypisanego roku wydania.

In [43]:
def display_book(book)
    "<h1>#{book.title} #{book.published}</h1><img src='#{book.author.image_url}'/>"
end
document = ""
books = Book.where("published is null")
books.each do |book|
  document += display_book(book)
end
IRuby.html document

### Zadanie 6

Zmodyfikuj funkcję `display_author` tak, aby wyświetlała również wszystkie książki napisane przez autora. Użyj znacznika `<ul>`.
Korzystając z tej funkcji wyświetl opis Stanisława Lema.

In [17]:
def display_author(author)
  document = "<h1>#{author.name} #{author.surname} #{author.born.strftime("%d.%m.%Y")} - #{author.died.strftime("%d.%m.%Y")} </h1>
  <hr />
  <img style='float:right;' src='#{author.image_url}'/>
  <ul>"
  books = author.books.order(:published)
  books.each do |book|
    document += "<li>#{book.title} - #{book.published}</li>"
  end
   document += "</ul>"
end
  
document = ""
author = Author.where(surname: "Lem").first
#dlaczego nie działa bez .first ?

document = display_author(author)
IRuby.html document




"<h1>Stanisław Lem 12.09.1921 - 27.03.2006 </h1>\n  <hr />\n  <img style='float:right;' src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/>\n  <ul><li>Sezam i inne opowiadania - 1954</li><li>Inwazja z Aldebarana (zbiór opowiadań) - 1959</li><li>Solaris (powieść) - 1961</li><li>Księga robotów - 1961</li><li>Pamiętnik znaleziony w wannie - 1961</li><li>Noc księżycowa - 1963</li><li>Bajki robotów - 1964</li><li>Niezwyciężony i inne opowiadania - 1964</li><li>Summa technologiae - 1964</li><li>Polowanie (zbiór opowiadań) - 1965</li><li>Głos Pana - 1968</li><li>Bezsenność (zbiór opowiadań) - 1971</li><li>Katar (powieść) - 1976</li><li>Maska (zbiór dzieł) - 1976</li><li>Powtórka (zbiór dzieł) - 1979</li><li>Wizja lokalna (powieść) - 1982</li><li>Pożytek ze smoka - 1993</li><li>Rasa drapieżców. Teksty ostatnie - 2006</li></ul>"

### Zadanie 7

Wśród książek napisanych przez Lema wyświetl wszystkie, które zawieraja w tytule słowo "zbiór".

In [12]:
def display_author(author)
  document = "<h1>#{author.name} #{author.surname} #{author.born.strftime("%d.%m.%Y")} - #{author.died.strftime("%d.%m.%Y")} </h1>
  <hr />
  <img style='float:right;' src='#{author.image_url}'/>
  <ul>"
  books = author.books.where("title like '%zbiór%'")
  books.each do |book|
    document += "<li>#{book.title}</li>"
  end
   document += "</ul>"
end
  
document = ""
author = Author.where(surname: "Lem").first
#dlaczego nie działa bez .first ?

document = display_author(author)
IRuby.html document

### Zadanie 8

Wśród książek napisanych przez Lema wyświetl te, które zostały wydane w roku jego śmierci.

In [3]:
def display_author(author)
  author_died_year = author.died.strftime("%Y")
  document = "<h1>#{author.name} #{author.surname} #{author.born.strftime("%d.%m.%Y")} - #{author.died.strftime("%d.%m.%Y")} </h1>
  <hr />
  <img style='float:right;' src='#{author.image_url}'/>
  <ul>"
  books = author.books.where(published: author_died_year)
  books.each do |book|
    document += "<li>#{book.title} - #{book.published}</li>"
  end
   document += "</ul>"
end
  
document = ""
author = Author.where(surname: "Lem").first
#dlaczego nie działa bez .first ?

document = display_author(author)
IRuby.html document


"<h1>Stanisław Lem 12.09.1921 - 27.03.2006 </h1>\n  <hr />\n  <img style='float:right;' src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/>\n  <ul><li>Rasa drapieżców. Teksty ostatnie - 2006</li></ul>"

### Zadanie 9

Wyświetl wszystkie książki znajdujące się w bazie. Opis powinien zawierać tytuł, imię i nazwisko autora, rok wydania oraz 
obraz przedstawiający książkę (o ile istnieje).

In [3]:
def display_book(book)
  document = "<hr />
  <strong>#{book.title}</strong><img src='#{book.image_url}'/>
  <ul>"
  document += "<li>#{book.author.name} #{book.author.surname} - #{book.published}</li>"
  document += "</ul>"
end
  
document = "<h1> Wszystkie książki: </h1>"
books = Book.all

books.each do |book|
  document += display_book(book)
end

IRuby.html document

"<h1> Wszystkie książki: </h1><hr />\n  <strong>Végjáték (regény)</strong><img src=''/>\n  <ul><li>Orson Scott Card - </li></ul><hr />\n  <strong>Pani Jeziora (powieść)</strong><img src=''/>\n  <ul><li>Andrzej Sapkowski - 1999</li></ul><hr />\n  <strong>Bellew Zawierucha</strong><img src=''/>\n  <ul><li>Jack London - 1912</li></ul><hr />\n  <strong>Fałszywy trop (powieść)</strong><img src=''/>\n  <ul><li>Henning Mankell - 1995</li></ul><hr />\n  <strong>Bajki robotów</strong><img src=''/>\n  <ul><li>Stanisław Lem - 1964</li></ul><hr />\n  <strong>Bolesław Chrobry (powieść)</strong><img src=''/>\n  <ul><li>Antoni Gołubiew - 1947</li></ul><hr />\n  <strong>De revolutionibus orbium coelestium</strong><img src=''/>\n  <ul><li>Mikołaj Kopernik - 1543</li></ul><hr />\n  <strong>Dzieło Wielkiego Miłosierdzia</strong><img src=''/>\n  <ul><li>Maria Franciszka Kozłowska - 1922</li></ul><hr />\n  <strong>Folwark zwierzęcy</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Animalism_flag.svg?width=300'/>\n  <ul><li>George Orwell - 1945</li></ul><hr />\n  <strong>Hamlet</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hamlet_quarto_3rd.jpg?width=300'/>\n  <ul><li>William Szekspir - </li></ul><hr />\n  <strong>Harry Potter</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Harry_Potter_wordmark.svg?width=300'/>\n  <ul><li>J. K. Rowling - 1997</li></ul><hr />\n  <strong>Hobbit, czyli tam i z powrotem</strong><img src=''/>\n  <ul><li>John Ronald Reuel Tolkien - 1937</li></ul><hr />\n  <strong>Hrabia Monte Christo</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Louis_Français-Dantès_sur_son_rocher.jpg?width=300'/>\n  <ul><li>Aleksander Dumas (ojciec) - 1844</li></ul><hr />\n  <strong>Kariera Nikodema Dyzmy</strong><img src=''/>\n  <ul><li>Tadeusz Dołęga-Mostowicz - 1932</li></ul><hr />\n  <strong>Krzyżacy (powieść)</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/PL_Henryk_Sienkiewicz-Krzyżacy_0005.jpeg?width=300'/>\n  <ul><li>Henryk Sienkiewicz - 1900</li></ul><hr />\n  <strong>Lolita</strong><img src=''/>\n  <ul><li>Vladimir Nabokov - 1955</li></ul><hr />\n  <strong>Mein Kampf</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Landsberg.Jail.jpg?width=300'/>\n  <ul><li>Adolf Hitler - 1925</li></ul><hr />\n  <strong>Nad Niemnem</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nad_Niemnem_-_manuscript.jpg?width=300'/>\n  <ul><li>Eliza Orzeszkowa - 1888</li></ul><hr />\n  <strong>Niedokończone opowieści</strong><img src=''/>\n  <ul><li>Christopher Tolkien - 1980</li></ul><hr />\n  <strong>Nowe Ateny</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nowe_ateny1.jpg?width=300'/>\n  <ul><li>Benedykt Chmielowski - 1745</li></ul><hr />\n  <strong>Nowy wspaniały świat</strong><img src=''/>\n  <ul><li>Aldous Huxley - 1932</li></ul><hr />\n  <strong>Ogniem i mieczem</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Podbipieta.JPG?width=300'/>\n  <ul><li>Henryk Sienkiewicz - 1884</li></ul><hr />\n  <strong>Ojciec Goriot</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Father_Goriot_by_H._Daumier_(1842).JPG?width=300'/>\n  <ul><li>Honoré de Balzac - 1835</li></ul><hr />\n  <strong>Proces (powieść)</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kafka_Der_Prozess_1925.jpg?width=300'/>\n  <ul><li>Franz Kafka - 1925</li></ul><hr />\n  <strong>Psałterz Dawidów</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kochanowski_-_Psałterz_Dawidów_(1579).jpg?width=300'/>\n  <ul><li>Jan Kochanowski - 1579</li></ul><hr />\n  <strong>Quo vadis</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Rzym_popiersie_Sienkiewicza_w_kosciele_Domine_Quo_Vadis.JPG?width=300'/>\n  <ul><li>Henryk Sienkiewicz - 1896</li></ul><hr />\n  <strong>Romeo i Julia</strong><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Romeo_and_jul

### Zadanie 10

Wyświetl wszystkich autorów znajdujących się w bazie wraz z opisem zawierającym ich zdjęcie, datę urodzin i śmierci oraz spis 
opublikowanych książek. Pamiętaj o odpowiednim sformatowaniu dat oraz o tym, że nie wszyscy autorzy mają przypisane wszystkie
informacje.

In [14]:

def display_author(author)
  document = "<hr /><strong>#{author.name} #{author.surname}</strong><br />"
  
  if author.born  
    document += "ur. #{author.born.strftime('%d.%m.%Y')}"
  end
  if author.born && author.died
    document += " - "
  end
  if author.died 
    document += "zm. #{author.died.strftime('%d.%m.%Y')}"
  end

  document += "<img src='#{author.image_url}'/>
  <ul>"
    books = author.books.order(:published)
  books.each do |book|
    document += "<li>#{book.title} - #{book.published}</li>"
  end
   document += "</ul>"
end
  
document = "<h1> Autorzy: </h1>"
authors = Author.all
authors.each do |author|
  document += display_author(author)
end

IRuby.html document

"<h1> Autorzy: </h1><hr /><strong>Juliusz Słowacki</strong><br />ur. 04.09.1809 - zm. 03.04.1849<img src=''/>\n  <ul></ul><hr /><strong>Henryk Sienkiewicz</strong><br />ur. 05.05.1846 - zm. 05.11.1916<img src=''/>\n  <ul><li>Ogniem i mieczem - 1884</li><li>Quo vadis - 1896</li><li>Krzyżacy (powieść) - 1900</li></ul><hr /><strong>Eliza Orzeszkowa</strong><br />ur. 06.06.1841 - zm. 18.05.1910<img src=''/>\n  <ul><li>Nad Niemnem - 1888</li></ul><hr /><strong>Adam Mickiewicz</strong><br />ur. 24.12.1798 - zm. 26.11.1855<img src=''/>\n  <ul><li>Księgi narodu polskiego i pielgrzymstwa polskiego - 1832</li></ul><hr /><strong>Orson Scott Card</strong><br />ur. 24.08.1951<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>\n  <ul><li>Végjáték (regény) - </li></ul><hr /><strong>Andrzej Sapkowski</strong><br />ur. 21.06.1948<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/>\n  <ul><li>Pani Jeziora (powieść) - 1999</li></ul><hr /><strong>Jack London</strong><br />ur. 12.01.1876 - zm. 22.11.1916<img src='http://commons.wikimedia.org/wiki/Special:FilePath/JackLondon02.jpeg?width=300'/>\n  <ul><li>Bellew Zawierucha - 1912</li></ul><hr /><strong>Henning Mankell</strong><br />ur. 03.02.1948<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Henning_Mankell_3_2011_Shankbone.jpg?width=300'/>\n  <ul><li>Fałszywy trop (powieść) - 1995</li></ul><hr /><strong>Stanisław Lem</strong><br />ur. 12.09.1921 - zm. 27.03.2006<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/>\n  <ul><li>Sezam i inne opowiadania - 1954</li><li>Inwazja z Aldebarana (zbiór opowiadań) - 1959</li><li>Solaris (powieść) - 1961</li><li>Księga robotów - 1961</li><li>Pamiętnik znaleziony w wannie - 1961</li><li>Noc księżycowa - 1963</li><li>Bajki robotów - 1964</li><li>Niezwyciężony i inne opowiadania - 1964</li><li>Summa technologiae - 1964</li><li>Polowanie (zbiór opowiadań) - 1965</li><li>Głos Pana - 1968</li><li>Bezsenność (zbiór opowiadań) - 1971</li><li>Katar (powieść) - 1976</li><li>Maska (zbiór dzieł) - 1976</li><li>Powtórka (zbiór dzieł) - 1979</li><li>Wizja lokalna (powieść) - 1982</li><li>Pożytek ze smoka - 1993</li><li>Rasa drapieżców. Teksty ostatnie - 2006</li></ul><hr /><strong>Antoni Gołubiew</strong><br />ur. 25.02.1907 - zm. 27.06.1979<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Antoni_Golubiew.jpg?width=300'/>\n  <ul><li>Bolesław Chrobry (powieść) - 1947</li></ul><hr /><strong>Mikołaj Kopernik</strong><br />ur. 19.02.1473 - zm. 24.05.1543<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300'/>\n  <ul><li>De revolutionibus orbium coelestium - 1543</li></ul><hr /><strong>Maria Franciszka Kozłowska</strong><br />ur. 27.05.1862<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Mateczk_Kozłowska.JPG?width=300'/>\n  <ul><li>Dzieło Wielkiego Miłosierdzia - 1922</li></ul><hr /><strong>George Orwell</strong><br />ur. 25.06.1903 - zm. 21.01.1950<img src='http://commons.wikimedia.org/wiki/Special:FilePath/GeoreOrwell.jpg?width=300'/>\n  <ul><li>Droga na molo w Wigan - 1937</li><li>Folwark zwierzęcy - 1945</li></ul><hr /><strong>William Szekspir</strong><br />ur. 23.04.1564 - zm. 03.05.1616<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Shakespeare.jpg?width=300'/>\n  <ul><li>Hamlet - </li><li>Romeo i Julia - 1597</li><li>Sen nocy letniej - 1600</li></ul><hr /><strong>J. K. Rowling</strong><br />ur. 31.07.1965<img src='http://commons.wikimedia.org/wiki/Special:FilePath/J._K._Rowling_2010.jpg?width=300'/>\n  <ul><li>Harry Potter - 1997</li></ul><hr /><strong>John Ronald Reuel Tolkien</strong><br />ur. 03.01.1892 - zm. 02.09.1973<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Tolkien_1916.jpg?width=300'/>\n  <ul><li>Hobbit, czyli tam i z powrotem - 1937</li><li>Władca Pierścieni - 1954</li><li>Silmarillion - 1977</li></ul><hr 